In [5]:
from typing import Optional, List, Dict, Generator, Any, Tuple
from collections import defaultdict, deque


import requests
from pydantic import HttpUrl

from srl_notion.api_connection import NotionAPIConnector
from srl_notion.api_result_fetcher import ResultFetcher
from srl_notion import utils

connector = NotionAPIConnector()
raw_response:dict = connector.main()
headers = connector.headers

fetcher = ResultFetcher(raw_response=raw_response, headers=headers)

all_pages:List[Tuple] = [res for res in fetcher.main()]

In [6]:
all_pages

[('Ton per',
  {'OMG une page ! (1)': 'https://www.notion.so/OMG-une-page-1-14c3cb01ecb04518a3641291e6e4d109'}),
 ('Ton per',
  {'Truc (1)': 'https://www.notion.so/Truc-1-604d0ff2fe544013bb6708b58f8ba65c'}),
 ('ta mere',
  {'ta mere': 'https://www.notion.so/ta-mere-80497c9312324710b14db45a27563932'}),
 ('Page test',
  {'Propal Raphael ': 'https://www.notion.so/Propal-Raphael-e266eee4b5134c48a7937dfa05b5240f'}),
 ('Page test',
  {'ma bite t': 'https://www.notion.so/ma-bite-t-7c547acf50de431a8b9559858a152330'}),
 ('Page test',
  {'Machin': 'https://www.notion.so/Machin-ee95162e13c349f38ea78253611910d9'})]

In [11]:
def prepare_output(fetcher_results:List[Tuple[Dict[str, str]]]) -> defaultdict[str, deque]:
    
    defdict_deque = defaultdict(deque)
    
    subject:str
    dict_page:Dict[str, HttpUrl]
    for subject, dict_page in fetcher_results:
        defdict_deque[tuple_[0]].append(tuple_[1])
        
    return defdict_deque
        
prepare_output(fetcher_results=all_pages)

defaultdict(collections.deque,
            {'Ton per': deque([{'OMG une page ! (1)': 'https://www.notion.so/OMG-une-page-1-14c3cb01ecb04518a3641291e6e4d109'},
                    {'Truc (1)': 'https://www.notion.so/Truc-1-604d0ff2fe544013bb6708b58f8ba65c'}]),
             'ta mere': deque([{'ta mere': 'https://www.notion.so/ta-mere-80497c9312324710b14db45a27563932'}]),
             'Page test': deque([{'Propal Raphael ': 'https://www.notion.so/Propal-Raphael-e266eee4b5134c48a7937dfa05b5240f'},
                    {'ma bite t': 'https://www.notion.so/ma-bite-t-7c547acf50de431a8b9559858a152330'},
                    {'Machin': 'https://www.notion.so/Machin-ee95162e13c349f38ea78253611910d9'}])})

In [12]:
def prepare_output(fetcher_results: List[Tuple[Dict[str, str]]]) -> defaultdict[str, deque]:
    defdict_deque = defaultdict(deque)
    for k, v in fetcher_results:
        defdict_deque[k].append(v)
    return defdict_deque


prepare_output(fetcher_results=all_pages)

defaultdict(collections.deque,
            {'Ton per': deque([{'OMG une page ! (1)': 'https://www.notion.so/OMG-une-page-1-14c3cb01ecb04518a3641291e6e4d109'},
                    {'Truc (1)': 'https://www.notion.so/Truc-1-604d0ff2fe544013bb6708b58f8ba65c'}]),
             'ta mere': deque([{'ta mere': 'https://www.notion.so/ta-mere-80497c9312324710b14db45a27563932'}]),
             'Page test': deque([{'Propal Raphael ': 'https://www.notion.so/Propal-Raphael-e266eee4b5134c48a7937dfa05b5240f'},
                    {'ma bite t': 'https://www.notion.so/ma-bite-t-7c547acf50de431a8b9559858a152330'},
                    {'Machin': 'https://www.notion.so/Machin-ee95162e13c349f38ea78253611910d9'}])})